# 프로젝트 가보

In [1]:
from flask import Flask, render_template, redirect, request, url_for, session
import cx_Oracle as db

app = Flask(__name__)

db_id = 'parky'
db_pw = 'qwer1234'
url = '192.168.21.252:1521/xe'

@app.route('/join', methods=['POST'])
def join():
    print('회원가입 요청')
    id = request.form['id']
    pw = request.form['pw']
    email = request.form['email']
    name = request.form['name']
    nick = request.form['nick']
    phone = request.form['phone']
    birth = request.form['birth']
    gender = request.form['gender']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "insert into t_user(user_id, user_pw,user_name,user_nick,user_birth,user_phone,user_email,user_gender,user_startdate) values (:1,:2,:3,:4,:5,:6,:7,:8,sysdate)"
    curs.execute(sql,(id,pw,name,nick,birth,phone,email,gender))
    conn.commit()
    curs.close()
    conn.close()
    print(f'회원정보 : {id},{pw},{name},{nick},{birth},{phone},{email},{gender}')
    return '가입 완료'

@app.route('/login', methods=['POST'])
def login():
    print('로그인 시도')
    id = request.form['id']
    pw = request.form['pw']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "select * from t_user where (user_id = :1) and (user_pw = :2)"
    curs.execute(sql,(id,pw))
    result = curs.fetchall()
    print(result)
    curs.close()
    conn.close()
    try :
        ##print("{},{},{},{},{},{},{},{},{},{},{},{}".format(result[0][0],result[0][1],result[0][2],result[0][3].isoformat()[:10],result[0][4],result[0][5],result[0][6],result[0][7],result[0][8].isoformat()[:10],result[0][10],result[0][11],result[0][13]))
        print("{} 로그인".format(result[0][0]))
    
        if result[0][12] == 1:
            ## 블랙리스트
            return "블랙리스트"
        elif result[0][9] is not None:
            return "탈퇴회원"
        else :
            return "로그인성공"
            ##return "{},{},{},{},{},{},{},{},{},{},{},{}".format(result[0][0],result[0][1],result[0][2],result[0][3].isoformat()[:10],result[0][4],result[0][5],result[0][6],result[0][7],result[0][8].isoformat()[:10],result[0][10],result[0][11],result[0][13])
    except :
        return "회원아님"

@app.route('/end', methods=['POST'])
def end():
    print('탈퇴 요청')
    id = request.form['id']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "update t_user set user_enddate = sysdate where user_id = :1"
    curs.execute(sql(id))
    curs.commit()
    curs.close()
    conn.close()
    
    return "탈퇴"

## 마이페이지 = 숨긴보물수, 찾은보물수, 좋아요, 레벨 갱신
@app.route('/mypage', methods=['POST'])
def mypage():
    print('보물 갱신')
    id = request.form['id']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "select (select count(t.t_hideuser) from t_treasure t where t.t_hideuser = :1), (select count(t.t_finduser) from t_treasure t where t.t_finduser = :1) , (select user_like from user where t_hideuser = :1), (select user_level from user where t_hideuser = :1) from t_treasure where t_hideuser = :1 GROUP BY t_hideuser"
    curs.execute(sql,(id))
    result = curs.fetchall()
    
    curs.close()
    conn.close()
    
    ## 숨긴보물수, 찾은보물수, 좋아요, 레벨 순
    return "{},{},{},{}".format(result[0][0],result[0][1],result[0][2],result[0][3])

@app.route('/alluserlist', methods=['POST'])
def alluserlist():
    print('모든 유저 리스트 조회')
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "select * from user"
    curs.execute(sql)
    result = curs.fetchall()
    print(result)
    curs.close()
    conn.close()
    
    listmember = ""
    
    for i in range(len(result)):
        for j in range(len(result[0])):
            if j ==3:
                listmember += str(result[i][j].isoformat()[:10]) + ","
            elif j ==13:
                listmember += str(result[i][j]) + "/"
            else:
                listmember += str(result[i][j]) + ","
        
        
    return listmember

@app.route('/addtreasure', methods=['POST'])
def addtreasure():
    print('보물 등록')
    cate = request.form['cate'] ## 코드(숫자)
    key1 = request.form['key1']
    key2 = request.form['key2']
    key3 = request.form['key3']
    loca = request.form['loca'] ## 장소 코드?? 
    hideuser = request.form['hideuser']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    ## :1이랑 :5가 넘버타입이라 인서트 안될수도 있음 그러면 f포멧팅 형식으로 바꿔보기 그래도 안되면 위 변수들을 int()로 감싸주기
    
    sql = "insert into t_treasure values(1,:1,:2,:3,:4,:5,:6,'','',0)"
    curs.execute(sql(cate,key1,key2,key3,loca,hideuser))
    curs.commit()
    curs.close()
    conn.close()
    
    return "등록완료"
    
#### 관리자 페이지
@app.route('/', methods=['GET'])
def loginPage():
    return render_template('page-login.html')

@app.route('/loginCheck', methods = ['POST']) #로그인 기능
def loginCheck():
    print('로그인 요청')
    admin_id = request.form['id']
    admin_pw = request.form['pw']
    
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select * from t_admin where admin_id = :1 and admin_pw = :2"
    curs.execute(sql, (admin_id, admin_pw))
    result = curs.fetchall();

    curs.close()
    conn.close()

    if admin_id == result[0][0] and admin_pw == result[0][1]:
        session['id'] = admin_id
        return redirect(url_for('mainPage'))
    else:
        return redirect(url_for('/'))




@app.route('/mainPage', methods = ['Get'])
def index():
    return render_template('index.html')   
          
if __name__ == '__main__':
    app.run(host='192.168.21.252', port=5013)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.21.252:5013/ (Press CTRL+C to quit)
192.168.21.252 - - [29/Aug/2022 15:11:07] "POST /index HTTP/1.1" 200 -
192.168.21.252 - - [29/Aug/2022 15:11:08] "GET /static/plugins/chartist/css/chartist.min.css HTTP/1.1" 200 -
192.168.21.252 - - [29/Aug/2022 15:11:08] "GET /static/plugins/pg-calendar/css/pignose.calendar.min.css HTTP/1.1" 200 -
192.168.21.252 - - [29/Aug/2022 15:11:08] "GET /static/plugins/chartist-plugin-tooltips/css/chartist-plugin-tooltip.css HTTP/1.1" 200 -
192.168.21.252 - - [29/Aug/2022 15:11:08] "GET /static/plugins/circle-progress/circle-progress.min.js HTTP/1.1" 200 -
192.168.21.252 - - [29/Aug/2022 15:11:08] "GET /static/plugins/topojson/topojson.min.js HTTP/1.1" 200 -
192.168.21.252 - - [29/Aug/2022 15:11:08] "GET /static/plugins/morris/morris.min.js HTTP/1.1" 200 -
192.168.21.252 - - [29/Aug/2022 15:11:08] "GET /static/plugins/raphael/raphael.min.js HTTP/1.1" 200 -
192.168.21.252 - - [29/Aug/2022 15:11:08] "GET /static/plugins/d3v3/index.j

로그인 시도
[('test1', 'qwer1234!', 'test', datetime.datetime(1991, 4, 13, 0, 0), '158745651231', 'wnefoxcn@naovn.sdfnm', '남', 1, datetime.datetime(2022, 8, 26, 9, 56, 12), None, None, 0, 0, 'test')]
test1 로그인


192.168.21.8 - - [29/Aug/2022 15:23:17] "POST /login HTTP/1.1" 200 -


로그인 시도
[('test1', 'qwer1234!', 'test', datetime.datetime(1991, 4, 13, 0, 0), '158745651231', 'wnefoxcn@naovn.sdfnm', '남', 1, datetime.datetime(2022, 8, 26, 9, 56, 12), None, None, 0, 0, 'test')]
test1 로그인


...

로그인 시도
[('test1', 'qwer1234!', 'test', datetime.datetime(1991, 4, 13, 0, 0), '158745651231', 'wnefoxcn@naovn.sdfnm', '남', 1, datetime.datetime(2022, 8, 26, 9, 56, 12), None, None, 0, 0, 'test')]
test1 로그인
로그인 시도
[('test1', 'qwer1234!', 'test', datetime.datetime(1991, 4, 13, 0, 0), '158745651231', 'wnefoxcn@naovn.sdfnm', '남', 1, datetime.datetime(2022, 8, 26, 9, 56, 12), None, None, 0, 0, 'test')]
test1 로그인


192.168.21.160 - - [29/Aug/2022 15:36:21] "POST /index HTTP/1.1" 200 -
192.168.21.160 - - [29/Aug/2022 15:36:21] "GET /static/plugins/pg-calendar/css/pignose.calendar.min.css HTTP/1.1" 200 -
192.168.21.160 - - [29/Aug/2022 15:36:21] "GET /static/plugins/chartist/css/chartist.min.css HTTP/1.1" 200 -
192.168.21.160 - - [29/Aug/2022 15:36:21] "GET /static/images/star.png HTTP/1.1" 200 -
192.168.21.160 - - [29/Aug/2022 15:36:21] "GET /static/images/logo-compact.png HTTP/1.1" 200 -
192.168.21.160 - - [29/Aug/2022 15:36:21] "GET /static/plugins/chartist-plugin-tooltips/css/chartist-plugin-tooltip.css HTTP/1.1" 200 -
192.168.21.160 - - [29/Aug/2022 15:36:21] "GET /static/images/avatar/1.jpg HTTP/1.1" 200 -
192.168.21.160 - - [29/Aug/2022 15:36:21] "GET /static/images/avatar/2.jpg HTTP/1.1" 200 -
192.168.21.160 - - [29/Aug/2022 15:36:21] "GET /static/images/avatar/3.jpg HTTP/1.1" 200 -
192.168.21.160 - - [29/Aug/2022 15:36:21] "GET /static/images/users/8.jpg HTTP/1.1" 200 -
192.168.21.160 - - 

로그인 시도
[]


192.168.21.8 - - [29/Aug/2022 15:37:22] "POST /login HTTP/1.1" 200 -


로그인 시도
[('test1', 'qwer1234!', 'test', datetime.datetime(1991, 4, 13, 0, 0), '158745651231', 'wnefoxcn@naovn.sdfnm', '남', 1, datetime.datetime(2022, 8, 26, 9, 56, 12), None, None, 0, 0, 'test')]
test1 로그인


In [3]:
!pip install cx_Oracle